In [6]:
from qutip import * # qutip == 5.0.4
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# Hamiltonian
$$\begin{align}​
H(t) &= \sum_{i = j}^{n} \frac{\Omega_j(t)}{2} |1\rangle_j {}_j\langle r| + \text{h.c.} \\​
&+ \sum_{j <> k} B_{jk} |rr\rangle_{jk} {}_{jk}\langle rr|.​
\end{align}$$

In [7]:
# Define the parameters
N = 1000 # time steps
T = 1.0 # total time
t = np.linspace(0, T, N) # time array
Omega_max = 10

# the Two Qubit System with Infinte Blockade Strength

In [ ]:
n = 2 # number of qubits
dim = 3 # 3 levels for each qubit

qubit1 = Qobj(dim) # first qubit
qubit2 = Qobj(dim) # second qubit
Joint_state = tensor(qubit1, qubit2) # joint state of the two qubits

Omega = np.zeros(N) # Global Pulse so Omega_1 = Omega_2 = Omega


In [9]:
# 定义基矢：|01⟩, |0r⟩, |11⟩, |W⟩=(|1r⟩+|r1⟩)/√2
basis_states = [basis(4, i) for i in range(4)] 

def H_global(Omega_max, phi):
    # 耦合项系数
    sqrt2 = np.sqrt(2)
    H = np.array([
        [0, Omega_max/2 * np.exp(1j*phi), 0, 0],
        [Omega_max/2 * np.exp(-1j*phi), 0, 0, 0],
        [0, 0, 0, sqrt2*Omega_max/2 * np.exp(1j*phi)],
        [0, 0, sqrt2*Omega_max/2 * np.exp(-1j*phi), 0]
    ])
    return Qobj(H)

def compute_fidelity(U, phases, T, Omega_max):
    # 初始态 |ψ(0)⟩ = |10⟩ + |11⟩ → 此处简化为 |01⟩ + |11⟩
    psi0 = (basis(4,0) + basis(4,2)).unit()
    
    # 时间演化
    H = [H_global(Omega_max, phi) for phi in phases]
    tlist = np.linspace(0, T, len(phases))
    result = mesolve(H, psi0, tlist, [], [])
    psiT = result.states[-1]
    
    # 计算各基态的相位
    a01 = np.exp(-1j*theta) * psiT[0]
    a11 = np.exp(-1j*(2*theta + np.pi)) * psiT[2]
    
    # 保真度公式
    F = (abs(1 + 2*a01 + a11)**2 + 1 + 2*abs(a01)**2 + abs(a11)**2) / 20
    return 1 - F

def optimize_phases(T, Omega_max, num_segments=100):
    # 初始猜测：随机相位
    phases0 = np.random.uniform(0, 2*np.pi, num_segments)
    theta0 = 0.0  # 初始单比特相位
    
    # 优化目标函数
    res = minimize(
        lambda x: compute_fidelity(x[:-1], x[-1], T, Omega_max),
        np.append(phases0, theta0),
        method='L-BFGS-B',
        bounds=[(0, 2*np.pi)]*num_segments + [(None, None)]
    )
    return res.x[:-1], res.x[-1]  # 返回优化后的相位和θ

In [ ]:
T_list = np.linspace(7.5, 7.7, 50) * Omega_max
fidelities = []

for T in T_list:
    phases_opt, theta_opt = optimize_phases(T, Omega_max)
    fid = 1 - compute_fidelity(phases_opt, theta_opt, T, Omega_max)
    fidelities.append(fid)

# 找到最优T*
T_opt = T_list[np.argmin(fidelities)]
print(f"Optimal duration: TΩ_max = {T_opt/Omega_max:.3f}")

TypeError: 'numpy.float64' object is not iterable

: 